In [2]:
import glob
import pandas as pd
from pathlib import Path

### API

In [3]:
#  dataframe multi-thread
from pandarallel import pandarallel
# Initialization
pandarallel.initialize()
# df.parallel_apply(func, progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
from tqdm import tqdm
def df_map(df):
    names = df.columns
    k_list = df[df.columns[0]].values.tolist()
    v_list = df[df.columns[1]].values.tolist()
    d = {}
    for k, v in tqdm(zip(k_list, v_list)):
        d[k] = v
    return d

##### pickle

In [5]:
# import pickle
# with open('/nas/chenyi/datasets_cls/map/gallery/stem2height_gallery2mw202208.pickle', 'wb') as f:
#     pickle.dump(stem2width_gallery2m, f)

### Data Infomation

In [6]:
folder_name = 'gallery_clear'

In [7]:
root = Path('/nas/chenyi/datasets_cls/gallery_detect/gallery_csv')
det_root = Path(f'/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/{folder_name}/')

In [8]:
!ls $root

gall2m_df20220725.csv  gall2m_hw_df20220725.csv  july


#### Dataload

##### OSS path

In [9]:
!ls /nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/

clred_final.csv       clred_temp.csv	    dadetv4	    dirty_data
clred_labels_oss.csv  clred_temp_final.csv  data_cls.ipynb


In [10]:
%%time
# oss 图片信息
# bboxes_all_df = pd.read_csv('/home/chenyi/workspace/myPyModule99/gallery_manage/gallery_clr_pipeline/data_temp/bboxes_clred_temp.csv')
bboxes_all_df = pd.read_csv('/home/chenyi/workspace/myPyModule99/gallery_manage/gallery_clr_pipeline/data_temp/bboxes_all_df.csv')

# bboxes_all_df.to_csv('/home/chenyi/workspace/myPyModule99/gallery_manage/gallery_clr_pipeline/data_temp/bboxes_all_df.csv'), index=False)
# len(gall_df), 

CPU times: user 5.72 s, sys: 666 ms, total: 6.39 s
Wall time: 16.1 s


### df处理

In [11]:
bboxes_all_df.columns

Index(['label_pred', 'bx1', 'bx2', 'bx3', 'bx4', 'conf', 'crop_id', 'txt_sp'], dtype='object')

In [12]:
bboxes_all_df['stem'] = bboxes_all_df['txt_sp'].apply(lambda x: Path(x).stem)

In [13]:
bboxes_all_df['crop_path'] = bboxes_all_df.apply(lambda row: '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/gall7kv6/crops/{0}/{1}/{2}_{3}.jpg'.format(
                                   row['stem'][:2],
                                   row['stem'][2:4],
                                   row['stem'],
                                   row['crop_id']), axis=1)

In [14]:
%%time
bboxes_all_df['crop_exist'] = bboxes_all_df['crop_path'].parallel_apply(lambda x: Path(x).is_file())

CPU times: user 793 ms, sys: 371 ms, total: 1.16 s
Wall time: 2min 34s


In [15]:
bboxes_all_df.value_counts('crop_exist')

crop_exist
True     7570719
False      40214
dtype: int64

In [17]:
# bboxes_all_df.to_csv('/home/chenyi/workspace/myPyModule99/gallery_manage/gallery_clr_pipeline/data_temp/bboxes_all_df.csv', index=False)

#### 图片宽高

In [27]:
import pickle
with open('/nas/chenyi/datasets_cls/map/gallery/stem2width_gallery2m202208.pickle', 'rb') as f:
    stem2width = pickle.load(f)
with open('/nas/chenyi/datasets_cls/map/gallery/stem2height_gallery2m202208.pickle', 'rb') as f:
    stem2height = pickle.load(f)

In [30]:
gall_old_df['width'] = gall_old_df['stem'].map(stem2width)
gall_old_df['height'] = gall_old_df['stem'].map(stem2height)

In [35]:
gall_all_df = pd.concat([gall_df, gall_df], axis=0)
gall_all_df.drop_duplicates('stem', inplace=True)

In [37]:
len(gall_all_df), len(gall_df)

(2193480, 2193480)

### 数据清洗

In [47]:
# state=0:未确认；state=1:确认干净； state=-1:脏数据
gall_df['state'] = 0

#### 图片尺寸

In [41]:
clred_df = gall_df 
len(clred_df)

2193480

##### 尺寸大小

In [43]:
thres = 300
temp = clred_df[(clred_df['width']<thres) | (clred_df['height']<thres)]
len(temp)

2322

In [44]:
clred_df = clred_df[~clred_df['stem'].isin(temp['stem'])]
len(temp)/len(clred_df)

0.0010597136308746335

In [45]:
thres = 350
temp = clred_df[(clred_df['width']<thres) & (clred_df['height']<thres)]
len(temp)

165

In [46]:
clred_df = clred_df[~clred_df['stem'].isin(temp['stem'])]
len(temp)/len(clred_df)

7.530831910462516e-05

In [59]:
gall_clred_df.describe()

,width,height,state
count,2.190993e+06,1.461834e+06,2190993.0
mean,8.654231e+02,1.137830e+03,0.0
std,2.981538e+02,3.745505e+02,0.0
min,3.000000e+02,3.000000e+02,0.0
25%,6.900000e+02,9.200000e+02,0.0
50%,6.900000e+02,1.035000e+03,0.0
75%,1.080000e+03,1.439000e+03,0.0
max,8.256000e+03,8.192000e+03,0.0


In [58]:
gall_clred_df = gall_df[gall_df['state']>=0]
len(gall_clred_df)

2190993

#### 数据标记

In [48]:
dirty_df = gall_df[~gall_df['stem'].isin(clred_df['stem'])]

In [49]:
gall_df.loc[dirty_df.index, 'state'] = -1

In [50]:
gall_df.value_counts('state')

state
 0    2190993
-1       2487
dtype: int64

##### 保存df

In [65]:
import json
with open('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/crops_rev_list.json', 'r') as f:
    crops_rev_list = json.load(f)

In [66]:
import json
with open('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/crops_imgs1m_list.json', 'r') as f:
    crops_1m_list = json.load(f)

In [67]:
len(crops_1m_list), len(crops_rev_list)

(4906804, 2302277)

In [69]:
crops_chg_list = [p.replace('dadetv4/gall_dadetv4_rev/crops/', 'dadetv4/gall_dadetv4_all/crops/') for p in crops_rev_list]

In [70]:
crops_rev_list[0], crops_chg_list[0]

('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_rev/crops/shoe/ffa73c89-b27a-4c6e-b377-c12b49ee0aec-441022_0.jpg',
 '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops/shoe/ffa73c89-b27a-4c6e-b377-c12b49ee0aec-441022_0.jpg')

In [71]:
crops_list = crops_1m_list + crops_chg_list

In [72]:
crops_df = pd.DataFrame()
crops_df['crops_p'] = crops_list

In [73]:
len(crops_df.drop_duplicates('crops_p')), len(crops_df)

(6605629, 7209081)

In [130]:
# crops_df.to_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/crops_all_df.csv', index=False)
crops_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/crops_all_df.csv')

In [129]:
crops_df.head(2).values

array([['/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops/shoe/69313640-af52-465e-88c8-c0a0099ca5c7-699544_0.jpg',
        'shoe'],
       ['/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops/shoe/69313640-af52-465e-88c8-c0a0099ca5c7-699544_2.jpg',
        'shoe']], dtype=object)

In [131]:
crops_df['class'] = crops_df['crops_p'].apply(lambda x: Path(x).parts[-2])

In [132]:
crops_df.value_counts('class')

class
shoe         2789456
upperbody    1390175
lowerbody     813196
bag           786924
wholebody     490417
skirt         335461
dtype: int64

In [50]:
input_list = [list(item) for item in zip(crops_rev_list, crops_list)]

In [12]:
%%time
import shutil
def copy_data_mkdir(plist, mode='move'):
    ps = Path(plist[0])
    pt = Path(plist[1])
    if not pt.parent.is_dir():
        pt.parent.mkdir(parents=True, exist_ok=True)
    if not pt.is_file():
        if mode=='move':
            Path.rename(ps, pt)
        else:
            shutil.copyfile(ps, pt)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs


In [ ]:
%%time           
run_task_multithread(input_list, copy_data_mkdir, 20)

In [19]:
det_root = Path('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4')

In [14]:
len(imgs_gall_df)

1661397

In [23]:
imgs_gall1m_df = imgs_gall_df[:int(1e6)]
imgs_gall_rev_df = imgs_gall_df[int(1e6):]

In [22]:
len(imgs_gall1m_df)

1000000

In [ ]:
%%time
import shutil
def copy_data_mkdir(plist, mode='copy'):
    ps = Path(plist[0])
    pt = Path(plist[1])
    if not pt.parent.is_dir():
        pt.parent.mkdir(parents=True, exist_ok=True)
    if not pt.is_file():
        if mode=='move':
            Path.rename(ps, pt)
        else:
            shutil.copyfile(ps, pt)
            
run_task_multithread(input_list, copy_data_mkdir)

In [186]:
def read_txt(p):
    with open(str(p), 'r') as f:
        label_lines = f.readlines()
    return label_lines

In [187]:
def get_bboxes_info(p):
    import numpy as np
    lines_list = []
    for ind,line in enumerate(read_txt(p)):
        line_list = line.split('\n')[0].split(' ')
        line_list.append(ind)
        line_list.append(p)
        lines_list.append(line_list)
    return lines_list

In [192]:
txts_gall_df.head(1)

,txt_path
0,/nas/chenyi/datasets_cls/gallery_detect/galler...


In [195]:
def get_bboxes_multithread(p):
    bboxes_info = get_bboxes_info(p)
    bboxes_all += bboxes_info

In [211]:
from tqdm import tqdm
bboxes_all = []
for p in tqdm(txts_gall_df['txt_path']):
    bboxes_info = get_bboxes_info(p)
    bboxes_all += bboxes_info
    # break

100%|████████████████████████████| 2189265/2189265 [19:53<00:00, 1834.50it/s]


In [212]:
# %%time
# global bboxes_all
# bboxes_all = []
# run_task_multithread(txts_gall_df['txt_path'][:10], get_bboxes_multithread, 20)

In [7]:
# bboxes_df = pd.read_csv('./data_temp/bboxes_all_df.csv')
# bboxes_df = pd.DataFrame(bboxes_all)

In [13]:
bboxes_df.head(1)

,label_pred,bx1,bx2,bx3,bx4,conf,crop_id,txt_sp
0,1,0.391667,0.750694,0.083333,0.123611,0.79473,0,/nas/chenyi/datasets_cls/gallery_detect/galler...


In [ ]:
bboxes_df['stem'] = stem

In [12]:
bboxes_df.columns

Index(['label_pred', 'bx1', 'bx2', 'bx3', 'bx4', 'conf', 'crop_id', 'txt_sp'], dtype='object')

In [11]:
name_stand = ['label_pred', 'bx1', 'bx2', 'bx3', 'bx4', 'conf','crop_id', 'txt_sp']
bboxes_df.rename(columns={bboxes_df.columns[0]:name_stand[0],
                          bboxes_df.columns[1]:name_stand[1],
                          bboxes_df.columns[2]:name_stand[2],
                          bboxes_df.columns[3]:name_stand[3],
                          bboxes_df.columns[4]:name_stand[4],
                          bboxes_df.columns[5]:name_stand[5],
                          bboxes_df.columns[6]:name_stand[6],
                          bboxes_df.columns[7]:name_stand[7],
                         },
                          inplace=True)

In [14]:
bboxes_df.to_csv('./data_temp/bboxes_all_df.csv', index=False)

In [76]:
imgs_df['stem'] = imgs_df['path'].apply(lambda x: Path(x).stem)

In [77]:
txt_df['stem'] = txt_df['path'].apply(lambda x: Path(x).stem)

In [88]:
imgs_cross = imgs_df[imgs_df['stem'].isin(txt_df['stem'])]

In [91]:
txt_cross = txt_df[txt_df['stem'].isin(imgs_df['stem'])]

In [80]:
imgs_df.sort_values('stem',inplace=True)
txt_df.sort_values('stem',inplace=True)

In [92]:
len(imgs_df),len(txt_df), len(imgs_cross), len(txt_cross)

(478504, 453275, 453275, 453275)

In [93]:
imgs_cross.sort_values('stem',inplace=True)
txt_cross.sort_values('stem',inplace=True)

/tmp/ipykernel_411860/859653631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imgs_cross.sort_values('stem',inplace=True)


In [94]:
watoo_df = pd.DataFrame()
watoo_df['img_p'] = imgs_cross['path'].values
watoo_df['label_p'] = txt_cross['path'].values

In [97]:
watoo_df.tail(2).values

array([['/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/牛仔裤/92/fff77871-de9f-4776-9424-f41b48cc0a49.jpg',
        '/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/labels/fff77871-de9f-4776-9424-f41b48cc0a49.txt'],
       ['/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/女衫类/2/fffadbff-226c-4108-9673-f7a41a24a53b.jpg',
        '/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/labels/fffadbff-226c-4108-9673-f7a41a24a53b.txt']],
      dtype=object)

In [98]:
imgs_nodet = imgs_df[~imgs_df['stem'].isin(txt_df['stem'])]

In [99]:
len(imgs_nodet)

25229

In [134]:
# imgs_nodet.to_csv('./data/watoo470k/imgs_nodet.csv', index=False)

In [119]:
p = imgs_nodet['path'].values[9]

In [13]:
p = txt_df.head(1).values[0][0]
!cat $p

2 0.505992 0.516117 0.985353 0.78036 0.810297


In [49]:
p = txt_df.head(10).values[-4][0]
!cat $p

3 0.506 0.923333 0.305333 0.153333 0.707341
2 0.464667 0.59 0.572 0.545333 0.958252


In [74]:
with open(p,'r') as f:
    txt = f.readlines()

In [75]:
txt

['3 0.506 0.923333 0.305333 0.153333 0.707341\n',
 '2 0.464667 0.59 0.572 0.545333 0.958252\n']

In [69]:
for item in txt:
    list1 = re.split(r'\n', item)
    list2 = list1[0].split(' ')
    

In [70]:
list2

['2', '0.464667', '0.59', '0.572', '0.545333', '0.958252']

In [31]:
import sys
sys.path.append('../')

In [36]:
from utils.plots.plots import draw_bbox
from utils.bbox.conversation import *
from utils.display.html import *

In [124]:
import sys
sys.path.append('../')
from utils.data.transfer import copy_data_mkdir, run_task_multithread

In [133]:
imgs_nodet.head(2).values

array([[PosixPath('/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/卫衣/150/005a2302-330b-4afc-ae19-5d0e513ee14a.jpg'),
        '005a2302-330b-4afc-ae19-5d0e513ee14a',
        '/nas/lichangjian/wardrobe_2/done_/卫衣/005a2302-330b-4afc-ae19-5d0e513ee14a.jpg'],
       [PosixPath('/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/牛仔裤/80/009dcf1b-1bb6-435e-b30c-0678ada49aee.jpg'),
        '009dcf1b-1bb6-435e-b30c-0678ada49aee',
        '/nas/lichangjian/wardrobe_2/done_/牛仔裤/009dcf1b-1bb6-435e-b30c-0678ada49aee.jpg']],
      dtype=object)

In [177]:
sroot = '/nas/lichangjian/wardrobe_2/done_'
troot = '/nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia/nodet_kfolder'

In [136]:
imgs_nodet['path'] = imgs_nodet['path'].apply(lambda x: Path(x))

/tmp/ipykernel_411860/2584459815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imgs_nodet['path'] = imgs_nodet['path'].apply(lambda x: Path(x))


In [137]:
imgs_nodet['img_sp'] = imgs_nodet['path'].apply(lambda x: '{0}/{1}/{2}'.format(sroot, x.parts[-3], x.name))

/tmp/ipykernel_411860/1833358304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imgs_nodet['img_sp'] = imgs_nodet['path'].apply(lambda x: '{0}/{1}/{2}'.format(sroot, x.parts[-3], x.name))


In [178]:
imgs_nodet['img_tp'] = imgs_nodet['path'].apply(lambda x: '{0}/{1}_{2}/{3}'.format(troot, folder2key[x.parts[-3]], x.parts[-3], x.name))

/tmp/ipykernel_411860/361264578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imgs_nodet['img_tp'] = imgs_nodet['path'].apply(lambda x: '{0}/{1}_{2}/{3}'.format(troot, folder2key[x.parts[-3]], x.parts[-3], x.name))


In [180]:
input_list[0]

['/nas/lichangjian/wardrobe_2/done_/卫衣/005a2302-330b-4afc-ae19-5d0e513ee14a.jpg',
 '/nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia/nodet_kfolder/10_卫衣/005a2302-330b-4afc-ae19-5d0e513ee14a.jpg']

In [181]:
%%time
import shutil
input_list = [list(item) for item in zip(imgs_nodet['img_sp'].values, imgs_nodet['img_tp'].values)]
def copy_data_mkdir(plist, mode='copy'):
    ps = Path(plist[0])
    pt = Path(plist[1])
    if not pt.parent.is_dir():
        pt.parent.mkdir(parents=True, exist_ok=True)
    if not pt.is_file():
        if mode=='move':
            Path.rename(ps, pt)
        else:
            shutil.copyfile(ps, pt)
            
run_task_multithread(input_list, copy_data_mkdir)

CPU times: user 5.52 s, sys: 31.3 s, total: 36.9 s
Wall time: 37.1 s
